In [1]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time

from sklearn.model_selection import train_test_split
#From my EEG package 
import run_expts

#Let me see as many results as I want to see
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [5]:
data_types = ['Wake' , 'N1' , 'N2' , 'N3' , 'REM']
expt_num = [1,2,3,4]
clf_list = ['ROCKET']

#Results dictionary structure : {model : {expt_type : {folder : results_df }}}
results_dict = {}
for clf in clf_list:
    results_dict[clf] = {1 : {} , 2 : {} , 3: {} , 4 : {}}

files_not_found = []

for num in [1,2,3,4]:
    for datatype in data_types:
        for clf in clf_list:
            try:
                save_path = 'Results/Results_3/' + 'expt_' + str(num) + '_' + datatype + '_raw_clfs_results.h5'
#                 file_name = 'expt_' + str(num) + '_' + datatype + '_raw_clfs_results.h5'
                results_df = pd.read_hdf(save_path , key='df', mode='r')
            except FileNotFoundError:
                files_not_found.append(save_path)
                continue
                
            res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)           
            results_dict[clf][num][datatype] = res_df

In [9]:
clf = 'ROCKET'

for num in [3,4]:
        print('Expt Type: ' + str(num))
        not_displayed = []
        for key in list(results_dict[clf][num].keys() ):
            df = results_dict[clf][num][key].copy()
            cols_to_drop = [col for col in df.columns if df[col].mean() < 0.40]
            df = df.drop(cols_to_drop, axis = 1 )
            if len(df.columns) < 2:
                not_displayed.append(key)
                continue
            print(key)
            display(df)
        print(not_displayed)

Expt Type: 3
Wake


,random_state,Prefrontal_ROCKET_acc,Occipital_ROCKET_acc
0,1,0.517359,0.389957
1,2,0.516667,0.443939


N1


,random_state,Prefrontal_ROCKET_acc,Left Central_ROCKET_acc,Left Temporal_ROCKET_acc
0,1,0.468373,0.446111,0.447063
1,2,0.363394,0.400049,0.434017


REM


...

['N2', 'N3']
Expt Type: 4
N1


,random_state,Prefrontal_ROCKET_acc,Left Central_ROCKET_acc,Left Temporal_ROCKET_acc
0,1,0.452262,0.417222,0.467500
1,2,0.403077,0.406398,0.397338


REM


,random_state,Prefrontal_ROCKET_acc,Frontal_ROCKET_acc,Left Frontal_ROCKET_acc,Right Frontal_ROCKET_acc,Parietal_ROCKET_acc,Left Parietal_ROCKET_acc,Right Parietal_ROCKET_acc
0,1,0.492866,0.42781,0.420885,0.413901,0.449517,0.407115,0.425628
1,2,0.464667,0.44533,0.429945,0.407433,0.497222,0.477885,0.501496


['Wake', 'N2', 'N3']


In [11]:
from sktime.classification.interval_based import CanonicalIntervalForest
from sktime.classification.interval_based import DrCIF

In [14]:
classifier_list  = [{'C_I_F': CanonicalIntervalForest()}, {'Dr_C_I_F': DrCIF()}]   

for clf in classifier_list:
    print(list(clf.keys())[0] )

C_I_F
Dr_C_I_F


In [15]:
clf_name = list(clf.keys())[0]
clf_name

'Dr_C_I_F'